# Script to convert csv to Linked Places

This script takes the CSV file that has been manipulated by OpenRefine and converts it to the Linked Places format that allows the Peripleo visualisation to display point data and run facets etc. 

You will need to have installed node & npm on your machine. To do this depends on your operating system.

1. Windows - [installation](https://learn.microsoft.com/en-us/windows/dev-environment/javascript/nodejs-on-windows)
2. Linux eg Ubuntu - [installation](https://www.digitalocean.com/community/tutorials/how-to-install-node-js-on-ubuntu-20-04)
3. OSX - [installation](https://prototype-kit.service.gov.uk/docs/install/node-mac)


# Step by step code breakdown

The code that converts the raw csv to [Linked Places](https://github.com/LinkedPasts/linked-places-format) format is relatively straightforward, and I am sure can be far better written than the below example lays
out. 

## Install the required npm libraries

First off you need to make sure your machine has a few libraries available. These are moment, wikimedia-commons-file-path (to enable you to get full paths for wikcommons images)


```bash
npm install moment
npm install wikimedia-commons-file-path
npm install papaparse
```
or you can just run 

```bash
npm install
```
Which will install the packages from package.json file which looks like this:

```json
{
    "name": "heritage-at-risk",
    "version": "1.0.0",
    "type": "module",
    "dependencies": {
        "moment": "^2.30.1",
        "papaparse": "^5.4.1",
        "wikimedia-commons-file-path": "^1.2.0"
    }
}
```

### Script breakdown

The script works like this. Firstly you need to import the libraries and functions:

```javascript
/**
 * This script processes a CSV file containing heritage site data, transforms it into GeoJSON format, 
 * and writes the resulting data to a JSON file. The script includes functions to generate GeoJSON 
 * objects, metadata, types, depictions, and links based on the provided data.
 *
 * @file /heritage-at-risk/scripts/transform-har-lp-enhanced.js
 * @requires fs - The Node.js File System module to read and write files.
 * @requires Papa - The PapaParse library to parse CSV data.
 * @requires commons - A module to generate Wikimedia Commons file paths.
 * @requires moment - The Moment.js library to handle date formatting.
 */
import fs, { link } from 'fs';
import Papa from 'papaparse';
import commons from 'wikimedia-commons-file-path';
import moment from 'moment';
```

Once these have been imported the script can begin to run. 

### Creating the JSON nodes

The script then has various segments to create the nodes of the JSON array - for this project's data visualisation we need to have:

1. An index function to create a single metadata descriptive node - getIndexing
2. A reusable place function to create a spatial node for each individual place added to the JSON array. - getPlace
3. A reusable types function to create linked data types for each individual place - getTypes
4. A reusable depiction function to create an image node for each individual place (in this project we have stayed with only one image per place, you can have 1:many - getDepiction
5. A reusable links function to create the linked data resources that point to the place - this can have as many as needed - getLink
6. A function to build the place node to add to the JSON array - buildFeature

Some of these functions have parameters attached. Let's go through these in turn. 

### Indexing function

This function is very basic, it adds a metadata json segment to the array once only which looks like this:

```json
    {
        "@context": "https://schema.org/",
        "@type": "Dataset",
        "name": "Heritage-at-Risk - Historic England",
        "description": "An enriched dataset of Heritage at Risk entries in England",
        "license": "https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/",
        "identifier": "https://www.planning.data.gov.uk/dataset/heritage-at-risk"
    }
```

The code to achieve this is:

```javascript
/**
 * Generates an indexing object for the Heritage-at-Risk dataset.
 *
 * @returns {Object} An object containing metadata for the Heritage-at-Risk dataset.
 * @returns {string} @returns.@context - The context URL for the schema.
 * @returns {string} @returns.@type - The type of the schema object.
 * @returns {string} @returns.name - The name of the dataset.
 * @returns {string} @returns.description - A description of the dataset.
 * @returns {string} @returns.license - The license URL for the dataset.
 * @returns {string} @returns.identifier - The identifier URL for the dataset.
 */
const getIndexing = () => {
    return  {
        "@context": "https://schema.org/",
        "@type": "Dataset",
        "name": "Heritage-at-Risk - Historic England",
        "description": "An enriched dataset of Heritage at Risk entries in England",
        "license": "https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/",
        "identifier": "https://www.planning.data.gov.uk/dataset/heritage-at-risk"
    }
}
```
In the above code we have hard coded in the values required and no parameters are required in this code 

### Geometry

The geometry for each place added to the array is very simple. The JSON object looks like this:

```json
 {
        "type": "Point",
        "coordinates": [
          -0.30496,
          51.513514
        ]
 }
```
The code to create this looks like this and takes two parameters - the latitude and longitude for the place being added to the array. 

```javascript
/**
 * Creates a GeoJSON Point object from longitude and latitude.
 *
 * @param {number|string} lon - The longitude of the point.
 * @param {number|string} lat - The latitude of the point.
 * @returns {Object} A GeoJSON Point object with the specified coordinates.
 */
const getPlace = (lon,lat) => {
    return {
        type: 'Point',
        coordinates: [ parseFloat(lon), parseFloat(lat) ]
    };
}
```

### Types

Adding types to the JSON array is also parameter driven. An example of the JSON notation for this segment of each place looks like this:

```json
"types": [
        {
          "identifier": "https://www.wikidata.org/wiki/Q16970",
          "label": "A Wikidata type: church building"
        },
        {
          "identifier": "https://www.wikidata.org/wiki/Q15700831",
          "label": "A Wikidata type: Grade II* listed building"
        }
      ]
```
And is driven by this function:

```javascript
/**
 * Generates an object containing types based on the provided properties.
 *
 * @param {Object} properties - The properties object.
 * @param {string} [properties.wikiInstanceOf] - The Wikidata instance identifier.
 * @param {string} [properties.wikidataEntityID] - The Wikidata entity identifier.
 * @param {string} [properties.heritage_category] - The heritage category.
 * @param {string} [properties.site_sub_type] - The site sub-type.
 * @returns {Object} An object containing the types with their identifiers and labels.
 */
const getTypes = (properties) => {
    const wikiInstanceOf = properties.wikiInstanceOf ? 'https://www.wikidata.org/wiki/' + properties.wikiInstanceOf : null;
    const wikidataEntityID  = properties.wikidataEntityID ? 'https://www.wikidata.org/wiki/' + properties.wikidataEntityID : null;
    const heritageCategory = properties.heritage_category;
    const siteSubType = properties.site_sub_type;
    const types = [];

    if (wikiInstanceOf && siteSubType) {
        types.push({
            identifier: wikiInstanceOf,
            label: 'A Wikidata type: ' + siteSubType
        });
    }

    if (wikidataEntityID && heritageCategory) {
        types.push({
            identifier: wikidataEntityID,
            label: 'A Wikidata type: ' + heritageCategory
        });
    }

    const flatTypes = types.reduce((all, type) => {
        return {
            ...all,
            types: [
            ...(all.types || []),
            {
                identifier: type.identifier,
                label: type.label
            }
            ]
        };
    }, {});
    return flatTypes;
}
```
In this script, I have decided to just use two different entity types and have attached the constants for the urls and then flattened the array out. 

### Adding a depiction

The JSON notation for a depiction looks like this:

```json
 "depictions": [
        {
          "@id": "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Christ_the_Saviour%2C_Ealing_Broadway_-_geograph.org.uk_-_1759007.jpg/800px-Christ_the_Saviour%2C_Ealing_Broadway_-_geograph.org.uk_-_1759007.jpg",
          "thumbnail": "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Christ_the_Saviour%2C_Ealing_Broadway_-_geograph.org.uk_-_1759007.jpg/800px-Christ_the_Saviour%2C_Ealing_Broadway_-_geograph.org.uk_-_1759007.jpg",
          "label": "A depiction of the heritage site sourced via Wikimedia Commons"
        }
      ],
```

The constant to deliver the depiction is as below:

```javascript
/**
 * Generates a depiction object for a heritage site based on the provided row data.
 *
 * @param {Object} row - The data row containing information about the heritage site.
 * @param {string} row.image_path_commons - The path to the image on Wikimedia Commons.
 * @returns {Object|undefined} An object containing depictions of the heritage site, or undefined if no image path is provided.
 */
const getDepiction = (row) => {
    if (!row.image_path_commons) return;
    const wikicommons = commons('File:' + row.image_path_commons, 800);
    return {
        depictions: [{
            '@id': wikicommons,
            thumbnail: wikicommons,
            label: 'A depiction of the heritage site sourced via Wikimedia Commons'
        }]
    };
}

```
In this, we are only getting images from wikicommons due to Historic England's restrictive license standpoint - hopefully this will change as part of the outcomes of digital strategy implementation. Hold your breath. 

This bit of code uses the excellent wikimedia-commons-file-path module to expand a filename obtained via reconciliation in openrefine to illustrate the place (not every place has one.) 

### Links

The JSON notation for a link node looks like this:

```json
"links": [
        {
          "identifier": "https://www.achurchnearyou.com/church/15689",
          "type": "seeAlso",
          "label": "A Church Near You entry 15689"
        },
        {
          "identifier": "https://britishlistedbuildings.co.uk/101079392",
          "type": "seeAlso",
          "label": "British Listed Building entry 101079392"
        },
        {
          "identifier": "https://www.wikidata.org/wiki/Q17550837",
          "type": "seeAlso",
          "label": "Wikidata entity Q17550837"
        },
        {
          "identifier": "https://historicengland.org.uk/listing/the-list/list-entry/1079392",
          "type": "seeAlso",
          "label": "Historic England NHLE number 1079392"
        },
        {
          "identifier": "https://commons.wikimedia.org/wiki/Category:Christ_the_Saviour%27s_church,_Ealing_Broadway",
          "type": "seeAlso",
          "label": "Wikimedia Commons Category"
        },
        {
          "identifier": "https://en.wikipedia.org/wiki/Christ_the_Saviour_Church,_Ealing",
          "type": "seeAlso",
          "label": "Wikipedia (English)"
        }
      ]
```

This bit of code is very hackneyed - it uses a variety of linked data identifiers and a base url to add links to:
1. Historic England's list
2. A Church near you
3. British Listed buildings
4. Wikidata
5. Wikicommons
6. Wikipedia

The constant takes the properties object and parses it to create each link and then the array is flattened. 

```javascript
/**
 * Generates a list of links based on the provided properties.
 *
 * @param {Object} properties - The properties object containing various IDs and information.
 * @param {string} [properties.churchnearyouID] - The ID for the "A Church Near You" entry.
 * @param {string} [properties.britishListedBuildingID] - The ID for the British Listed Building entry.
 * @param {string} [properties.wikidata] - The ID for the Wikidata entity.
 * @param {string} [properties.list_entry_number] - The list entry number for Historic England.
 * @param {string} [properties.wikicommonsCategoryID] - The ID for the Wikimedia Commons category.
 * @param {string} [properties.wikipediaENID] - The ID for the English Wikipedia entry.
 * @returns {Object} An object containing an array of link objects, each with an identifier, type, and label.
 */
const getLinks = (properties) => {
    const urls = {
        churchnearyouID: 'https://www.achurchnearyou.com/church/',
        britishListedBuildingID: 'https://britishlistedbuildings.co.uk/',
        wikidata: 'https://www.wikidata.org/wiki/',
        list_entry_number: 'https://historicengland.org.uk/listing/the-list/list-entry/',
        wikipediaENID: 'https://en.wikipedia.org/wiki/',
        wikicommonsCategoryID: 'https://commons.wikimedia.org/wiki/'
    };

    const labels = {
        churchnearyouID: 'A Church Near You entry ',
        britishListedBuildingID: 'British Listed Building entry ',
        wikidata: 'Wikidata entity ',
        list_entry_number: 'Historic England NHLE number ',
        wikipediaENID: 'Wikipedia (English)',
        wikicommonsCategoryID: 'Wikimedia Commons Category'
    };

    const links = Object.keys(urls).reduce((acc, key) => {
        if (properties[key]) {
            acc.push({
                identifier: urls[key] + properties[key],
                type: 'seeAlso',
                label: labels[key] + (key === 'wikipediaENID' || key === 'wikicommonsCategoryID' ? '' : properties[key])
            });
        }
        return acc;
    }, []);

    return { links };
}
```

###  Building a feature

The feature that is added to the array runs using 5 parameters - the record, the place, the longitude, the latitude and each row. 
Feed in this data and use each constant function to create a feature. 

```javascript
/**
 * Builds a feature object from the given parameters.
 *
 * @param {Object} record - The record object containing initial properties.
 * @param {string} place - The place name associated with the feature.
 * @param {string} lon - The longitude coordinate of the feature.
 * @param {string} lat - The latitude coordinate of the feature.
 * @param {Object} row - The row object containing additional data for the feature.
 * @returns {Object|undefined} The constructed feature object or undefined if place and lon are not provided.
 */
const buildFeature = (record, place, lon, lat, row) => {
    console.log(record)
    if (!place?.trim() && !lon?.trim())
        return;

    return {
        ...record,
        properties: {
            ...record.properties,
            place: place.trim(),
        },
        geometry: {
            ...getPlace(lon,lat)
        },
        
            ...getDepiction(row)
        ,
            ...getTypes(row) 
        ,
            ...getLinks(row)
        
    }
}

```

The code will then create a place node, which looks like this:

```json
{
      "@id": "https://historicengland.org.uk/advice/heritage-at-risk/search-register/list-entry/10025",
      "type": "Feature",
      "properties": {
        "title": "All Saints' Church",
        "source": "https://historicengland.org.uk/advice/heritage-at-risk/search-register/list-entry/10025",
        "listEntryNumber": "1396401",
        "designatedSiteName": "Church Of All Saints",
        "heritageCategory": "Grade II* listed building",
        "localPlanningAuthority": "Barnet",
        "siteType": "Religious ritual and funerary",
        "siteSubType": "church building",
        "county": "Greater London",
        "districtOrBorough": "Barnet",
        "parish": "",
        "parliamentaryConstituency": "Finchley and Golders Green",
        "region": "London and South East",
        "assessmentType": "Place of worship",
        "condition": "Fair",
        "principalVunerability": "",
        "trend": "",
        "ownership": "Religious organisation",
        "unitaryAuthority": "",
        "buildingName": "All Saints Church",
        "occupancyOrUse": "",
        "priority": "F",
        "priorityComment": "Repair scheme in progress and (where applicable) end use or user identified; or functionally redundant buildings with new use agreed but not yet implemented.",
        "previousPriority": "F",
        "designation": "Listed Place of Worship grade II",
        "locality": "East Finchley",
        "listEntryNumbers": "",
        "nationalPark": "",
        "streetName": "Durham Road",
        "vulnerability": "",
        "endDate": "",
        "entity": "7504598",
        "entryDate": "26/09/2024",
        "yearListed": "2011",
        "wikicommonsCategoryID": "Category:All_Saints%27_Church,_East_Finchley",
        "wikipediaENID": "All_Saints%27_Church,_East_Finchley",
        "wikiInstanceOf": "Q16970",
        "wikidataEntityID": "Q15700831",
        "place": "Greater London"
      },
      "descriptions": [
        {
          "value": "Heritage at Risk Entry: 1396401<br/>All Saints' Church<ul><li>Entry date: 26/09/2024</li><li>First listed: 07/01/2011</li><li>Assessment type: Place of worship</li><li>Condition: Fair</li><li>Ownership: Religious organisation</li><li>Building name: All Saints Church</li><li>Priority: F</li><li>Priority comment: Repair scheme in progress and (where applicable) end use or user identified; or functionally redundant buildings with new use agreed but not yet implemented.</li><li>Previous priority: F</li><li>Designation: Listed Place of Worship grade II</li><li>Locality: East Finchley</li></ul>"
        }
      ],
      "geometry": {
        "type": "Point",
        "coordinates": [
          -0.15973,
          51.591825
        ]
      },
      "depictions": [
        {
          "@id": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/All_Saints'_Church%2C_East_Finchley_05.jpg/800px-All_Saints'_Church%2C_East_Finchley_05.jpg",
          "thumbnail": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/All_Saints'_Church%2C_East_Finchley_05.jpg/800px-All_Saints'_Church%2C_East_Finchley_05.jpg",
          "label": "A depiction of the heritage site sourced via Wikimedia Commons"
        }
      ],
      "types": [
        {
          "identifier": "https://www.wikidata.org/wiki/Q16970",
          "label": "A Wikidata type: church building"
        },
        {
          "identifier": "https://www.wikidata.org/wiki/Q15700831",
          "label": "A Wikidata type: Grade II* listed building"
        }
      ],
      "links": [
        {
          "identifier": "https://www.achurchnearyou.com/church/15558",
          "type": "seeAlso",
          "label": "A Church Near You entry 15558"
        },
        {
          "identifier": "https://britishlistedbuildings.co.uk/101396401",
          "type": "seeAlso",
          "label": "British Listed Building entry 101396401"
        },
        {
          "identifier": "https://www.wikidata.org/wiki/Q27087428",
          "type": "seeAlso",
          "label": "Wikidata entity Q27087428"
        },
        {
          "identifier": "https://historicengland.org.uk/listing/the-list/list-entry/1396401",
          "type": "seeAlso",
          "label": "Historic England NHLE number 1396401"
        },
        {
          "identifier": "https://commons.wikimedia.org/wiki/Category:All_Saints%27_Church,_East_Finchley",
          "type": "seeAlso",
          "label": "Wikimedia Commons Category"
        },
        {
          "identifier": "https://en.wikipedia.org/wiki/All_Saints%27_Church,_East_Finchley",
          "type": "seeAlso",
          "label": "Wikipedia (English)"
        }
      ]
    },
```

## But how do you call the above?

The code constants that output the json place notation is driven by this bit of code, which is pretty inefficient and could be written completely. However it does the job and notes are added in below explaining what is happening.

```javascript

/**
 * Reads the content of the CSV file located at '../rawData/har-lp-ready-csv-enhanced.csv' 
 * and stores it in the `recordsCsv` variable.
 *
 * @constant {string} recordsCsv - The content of the CSV file as a UTF-8 encoded string.
 * @requires fs - The Node.js File System module to read the file.
 */
const recordsCsv = fs.readFileSync('../rawData/har-lp-ready-csv-enhanced.csv', { encoding: 'utf8' });

/**
 * Parses a CSV string into an array of objects using PapaParse.
 *
 * @param {string} recordsCsv - The CSV string to be parsed.
 * @returns {Object[]} An array of objects representing the parsed CSV records.
 */
const records = Papa.parse(recordsCsv, { header: true });

/**
 * Generates a GeoJSON feature object for a heritage site based on the provided data.
 *
 * @param {Object} record - The base record object containing properties of the heritage site.
 * @param {string} record['@id'] - The source URL of the heritage site.
 * @param {string} record.type - The type of the GeoJSON object, typically 'Feature'.
 * @param {Object} record.properties - The properties of the heritage site.
 * @param {string} record.properties.title - The title of the heritage site.
 * @param {string} record.properties.source - The source URL of the heritage site.
 * @param {string} record.properties.listEntryNumber - The list entry number for Historic England.
 * @param {string} record.properties.heritageCategory - The heritage category of the site.
 * @param {string} record.properties.localPlanningAuthority - The local planning authority.
 * @param {string} record.properties.siteType - The type of the site.
 * @param {string} record.properties.siteSubType - The subtype of the site.
 * @param {string} record.properties.county - The county where the site is located.
 * @param {string} record.properties.districtOrBorough - The district or borough where the site is located.
 * @param {string} record.properties.parish - The parish where the site is located.
 * @param {string} record.properties.parliamentaryConstituency - The parliamentary constituency.
 * @param {string} record.properties.region - The region where the site is located.
 * @param {string} record.properties.assessmentType - The type of assessment.
 * @param {string} record.properties.condition - The condition of the site.
 * @param {string} record.properties.principalVunerability - The principal vulnerability of the site.
 * @param {string} record.properties.trend - The trend of the site's condition.
 * @param {string} record.properties.ownership - The ownership of the site.
 * @param {string} record.properties.unitaryAuthority - The unitary authority.
 * @param {string} record.properties.buildingName - The name of the building.
 * @param {string} record.properties.occupancyOrUse - The occupancy or use of the site.
 * @param {string} record.properties.priority - The priority level of the site.
 * @param {string} record.properties.priorityComment - Comments on the priority level.
 * @param {string} record.properties.previousPriority - The previous priority level.
 * @param {string} record.properties.designation - The designation of the site.
 * @param {string} record.properties.locality - The locality of the site.
 * @param {string} record.properties.listEntryNumbers - The list entry numbers.
 * @param {string} record.properties.nationalPark - The national park where the site is located.
 * @param {string} record.properties.streetName - The street name where the site is located.
 * @param {string} record.properties.vulnerability - The vulnerability of the site.
 * @param {string} record.properties.endDate - The end date of the site's listing.
 * @param {string} record.properties.entity - The entity associated with the site.
 * @param {string} record.properties.entryDate - The entry date of the site's listing.
 * @param {string} record.properties.yearListed - The year the site was listed.
 * @param {string} record.properties.wikicommonsCategoryID - The Wikimedia Commons category ID.
 * @param {string} record.properties.wikipediaENID - The English Wikipedia entry ID.
 * @param {string} record.properties.wikiInstanceOf - The Wikidata instance of the site.
 * @param {string} record.properties.wikidataEntityID - The Wikidata entity ID.
 * @param {Array<Object>} record.descriptions - An array of description objects.
 * @param {string} record.descriptions[].value - The description of the heritage site.
 * @param {string} place - The place name associated with the heritage site.
 * @param {string} lon - The longitude coordinate of the heritage site.
 * @param {string} lat - The latitude coordinate of the heritage site.
 * @param {Object} row - The data row containing additional information about the heritage site.
 * @param {string} row.image_path_commons - The path to the image on Wikimedia Commons.
 * @param {string} row.wikiInstanceOf - The Wikidata instance of the site.
 * @param {string} row.wikidataEntityID - The Wikidata entity ID.
 * @param {string} row.heritage_category - The heritage category of the site.
 * @param {string} row.site_sub_type - The subtype of the site.
 * @param {string} row.churchnearyouID - The ID for the "A Church Near You" entry.
 * @param {string} row.britishListedBuildingID - The ID for the British Listed Building entry.
 * @param {string} row.wikidata - The ID for the Wikidata entity.
 * @param {string} row.list_entry_number - The list entry number for Historic England.
 * @param {string} row.wikicommonsCategoryID - The ID for the Wikimedia Commons category.
 * @param {string} row.wikipediaENID - The ID for the English Wikipedia entry.
 * @returns {Object|undefined} A GeoJSON feature object representing the heritage site, or undefined if place and lon are not provided.
 */
const features = records.data.map(row => {
    const {
        name: title, lat, lon, url: source, list_entry_number: listEntryNumber, heritage_category: heritageCategory,
        local_planning_authority: localPlanningAuthority, site_type: siteType, site_sub_type: siteSubType, county,
        district_or_borough: districtOrBorough, parish, parliamentary_constituency: parliamentaryConstituency, region,
        assessment_type: assessmentType, condition, principal_vunerability: principalVunerability, trend, ownership,
        unitary_authority: unitaryAuthority, building_name: buildingName, occupancy_or_use: occupancyOrUse, priority,
        priority_comment: priorityComment, previous_priority: previousPriority, designation, locality, list_entry_numbers: listEntryNumbers,
        national_park: nationalPark, street_name: streetName, vulnerability, end_date: endDate, entity, entry_date: entryDate,
        list_start_date, wikicommonsCategoryID, wikipediaENID, wikidataEntityID, wikiInstanceOf
    } = row;

    const place = county;
    const formatDate = dateString => moment(dateString, "YYYY-MM-DD").format("DD/MM/YYYY");
    const formattedDate = formatDate(list_start_date);
    const yearListed = moment(formattedDate, "DD/MM/YYYY").year().toString() || null;

    const fields = [
        { label: 'Entry date', value: entryDate },
        { label: 'First listed', value: list_start_date ? formattedDate : null },
        { label: 'Assessment type', value: assessmentType },
        { label: 'Condition', value: condition },
        { label: 'Principal vulnerability', value: principalVunerability },
        { label: 'Trend', value: trend },
        { label: 'Ownership', value: ownership },
        { label: 'Unitary authority', value: unitaryAuthority },
        { label: 'Building name', value: buildingName },
        { label: 'Occupancy or use', value: occupancyOrUse },
        { label: 'Priority', value: priority },
        { label: 'Priority comment', value: priorityComment },
        { label: 'Previous priority', value: previousPriority },
        { label: 'Designation', value: designation },
        { label: 'Locality', value: locality },
        { label: 'List entry numbers', value: listEntryNumbers }
    ];

    const description = `Heritage at Risk Entry: ${listEntryNumber}<br/>${title}<ul>` +
        fields.filter(field => field.value).map(field => `<li>${field.label}: ${field.value}</li>`).join('') +
        '</ul>'.trim().replace(/\n/g, '');
    
    const peripleoRecord = {
        '@id': source.trim(),
        type: 'Feature',
        properties: {
            title, source, listEntryNumber, heritageCategory, localPlanningAuthority, siteType, siteSubType, county,
            districtOrBorough, parish, parliamentaryConstituency, region, assessmentType, condition, principalVunerability,
            trend, ownership, unitaryAuthority, buildingName, occupancyOrUse, priority, priorityComment, previousPriority,
            designation, locality, listEntryNumbers, nationalPark, streetName, vulnerability, endDate, entity, entryDate,
            yearListed, wikicommonsCategoryID, wikipediaENID, wikiInstanceOf, wikidataEntityID
        },
        descriptions: [{ value: description }]
    };

    return buildFeature(peripleoRecord, place, lon, lat, row);
}).filter(Boolean);
const indexing = getIndexing();
const fc = {
    type: 'FeatureCollection',
    indexing,
    features
};

/**
 * Writes the GeoJSON FeatureCollection object to a JSON file located at '../docs/data/harLP.json'.
 *
 * @requires fs - The Node.js File System module to write the file.
 */
fs.writeFileSync('../docs/data/harLP.json', JSON.stringify(fc, null, 2), 'utf8');
```

## Running this script

To run this script, you just need to use the command line and do:

```bash
node ./scripts/transform-har-lp-enhanced.js 
```
Let it run and your linked places geojson file should be created. And then you can set up peripleo. 